In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import  expon, norm, gamma, beta, lognorm
from scipy.stats._continuous_distns import beta_gen, gamma_gen
from scipy import stats
from datetime import datetime, timedelta

# EDA

In [ ]:
zone1 = pd.read_csv('data/out_1.csv', usecols=["Datum", "Uhrzeit", "Masse [kg]", "Geschwindigkeit [m/s]"])
zone2 = pd.read_csv('data/out_2.csv', usecols=["Date", "Uhrzeit", "m [kg]", "v [m/s]"])
zone1.columns = ["date", "time", "kg", "m/s"]
zone2.columns = ["date", "time", "kg", "m/s"]

In [ ]:
pd.concat([zone1, zone2], axis=1, keys=['zone1', 'zone2']).describe()

In [ ]:
zone1.head()

## NAs

In [ ]:
# Check for NaN values
print(zone1.isna().sum())

# Check for zeros
print(zone1.eq(0).sum())

In [ ]:
# Drop lines with only NA values
zone1 = zone1.dropna(how='all')

In [ ]:
zone2.head()


In [ ]:
# Check for NaN values
print(zone2.isna().sum())

# Check for zeros
print(zone2.eq(0).sum())

In [ ]:
# Drop lines with only NA values
zone2 = zone2.dropna(how='all')


In [ ]:
# Set the value to the median in the row where 'kg' equals 0.0
# This rock is not removed from the data, because we do not have a lot of data and this might just have been an error in the measurement
zone2.loc[zone2['kg'] == 0.0, 'kg'] = zone2['kg'].median()

In [ ]:
pd.concat([zone1, zone2], axis=1, keys=['zone1', 'zone2']).describe()

# Visualization

In [ ]:
zone1.plot(x='kg', y='m/s', kind='scatter', c='red', label='zone1', title='Mass vs Velocity Zone 1')
plt.show()

In [ ]:
zone2.plot(x='kg', y='m/s', kind='scatter', c='blue', label='zone2', title='Mass vs Velocity Zone 2')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(zone1['kg'], zone1['m/s'], c='red', label='zone1')
ax.scatter(zone2['kg'], zone2['m/s'], c='blue', label='zone2')
ax.legend()
ax.axes.set_xlabel('m [kg]')
ax.axes.set_ylabel('v [m/s]')
plt.title('Mass vs Velocity in both Zones')
plt.show()

Zonen 1 und 2 sollten nicht gemischt werden, weil sonst sehr schwere steine generiert werden könnten die eine hohe Geschwindigkeit haben, was in den Aufzeichnungen aber nicht vorkomt (nicht gleiche Grundgesammtheit).

In [ ]:
zone1['kj'] = 0.5 * zone1['kg'] * (zone1['m/s']**2) /1000
zone2['kj'] = 0.5 * zone2['kg'] * (zone2['m/s']**2) /1000

In [ ]:
# Convert the 'date' and 'time' columns to datetime for zone1
zone1['datetime'] = pd.to_datetime(zone1['date'] + ' ' + zone1['time'])

# Calculate the time difference between each row and the row above, in hours then set the first row to the median of all rows
zone1['timediv h'] = (zone1['datetime'] - zone1['datetime'].shift()).fillna(pd.Timedelta(seconds=0))
zone1['timediv h'] = zone1['timediv h'].apply(lambda x: int(round(x.total_seconds() / 3600)))

# Set the first row to the median of all rows so we don't loose a value
zone1.loc[0, 'timediv h'] = zone1['timediv h'].median()

# Print the updated dataframe
print(zone1)

In [ ]:
# Convert the 'date' and 'time' columns to datetime for zone2
zone2['datetime'] = pd.to_datetime(zone2['date'] + ' ' + zone2['time'])

# Calculate the time difference between each row and the row above, in hours
zone2['timediv h'] = (zone2['datetime'] - zone2['datetime'].shift()).fillna(pd.Timedelta(seconds=0))
zone2['timediv h'] = zone2['timediv h'].apply(lambda x: int(round(x.total_seconds() / 3600)))

# Set the first row to the median of all rows so we don't loose a value
zone2.loc[0, 'timediv h'] = zone2['timediv h'].median()

# Print the updated dataframe
print(zone2)

In [ ]:
num_bins = 10  # Change this value to adjust the number of bins

# Plot the histograms for 'kg'
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.hist(zone1['kg'], bins=num_bins, color='red', label='Zone 1')
ax2.hist(zone2['kg'], bins=num_bins, color='blue', label='Zone 2')

ax1.set_xlabel('kg')
ax1.set_ylabel('Frequency')
ax1.legend()
ax2.set_xlabel('kg')
ax2.legend()
plt.show()

# Plot the histograms for 'kj'
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.hist(zone1['kj'], bins=num_bins, color='red', label='Zone 1')
ax2.hist(zone2['kj'], bins=num_bins, color='blue', label='Zone 2')

ax1.set_xlabel('kj')
ax1.set_ylabel('Frequency')
ax1.legend()
ax2.set_xlabel('kj')
ax2.legend()
plt.show()

# Plot the histograms for 'm/s'
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.hist(zone1['m/s'], bins=num_bins, color='red', label='Zone 1')
ax2.hist(zone2['m/s'], bins=num_bins, color='blue', label='Zone 2')

ax1.set_xlabel('m/s')
ax1.set_ylabel('Frequency')
ax1.legend()
ax2.set_xlabel('m/s')
ax2.legend()
plt.show()

# Plot the histograms for 'timediv h'
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.hist(zone1['timediv h'], bins=num_bins, color='red', label='Zone 1')
ax2.hist(zone2['timediv h'], bins=num_bins, color='blue', label='Zone 2')

ax1.set_xlabel('timediv h')
ax1.set_ylabel('Frequency')
ax1.legend()
ax2.set_xlabel('timediv h')
ax2.legend()
plt.show()

In [ ]:
pd.concat([zone1, zone2], axis=1, keys=['zone1', 'zone2']).describe()

In [ ]:
def fit_best_distribution(data):
    # Define candidate distributions
    dist_names = [norm, lognorm, expon, gamma, beta]

    # Set up initial best parameters and likelihoods
    best_dist = None
    best_params = {}
    best_ll = 1_000_000

    # Set up a dictionary to store the log-likelihoods of each distribution
    ll_dict = {}

    # Iterate through candidate distributions and find the best fit
    for dist_name in dist_names:
        # Fit the distribution to the data using MLE
        params = dist_name.fit(data)

        # Get the negative log-likelihood of the data under the distribution
        ll = -dist_name.logpdf(data, *params).sum()

        # If the fit is better than the current best, update the best fit
        if ll < best_ll:
            best_dist = dist_name
            best_params = params
            best_ll = ll

        # Store the log-likelihood of the fit for this distribution
        ll_dict[dist_name] = ll

    # Sort the distributions by the log-likelihood of their fit
    sorted_dists = sorted(ll_dict, key=ll_dict.get)

    # Generate a list of strings describing the fit of each distribution
    dist_strings = []
    for dist_name in sorted_dists:
        if dist_name == best_dist:
            dist_strings.append(f'{dist_name} (best fit): {ll_dict[dist_name]}')
        else:
            dist_strings.append(f'{dist_name}: {ll_dict[dist_name]}')

    # Return the best distribution, its parameters, and the list of distribution fit strings
    return best_dist, best_params, dist_strings

In [ ]:
z1_kg_dist, z1_kg_param, z1_kg_results = fit_best_distribution(zone1['kg'])
z1_kg_results

# TODO: Plot the distribution of the best fit with the distribution of the data or qqplot

In [ ]:
fit_best_distribution(zone2['kg']) 

In [ ]:
fit_best_distribution(zone1['m/s'])

In [ ]:
fit_best_distribution(zone2['m/s'])

In [ ]:
fit_best_distribution(zone1['timediv h'])

In [ ]:
fit_best_distribution(zone1['timediv h'])

In [ ]:
# to simulate a dataframe for the next number of years it will estimate the number of events it takes and generate a frame 
def generate_simulated_data(zone_df, num_years=200):
    
    timediv_mean = zone_df['timediv h'].mean()
    
    # Calculate  the deviation
    timediv_params = gamma.fit(zone_df['timediv h'])
    kg_params = beta.fit(zone_df['kg'])
    ms_mean, ms_std = zone_df['m/s'].mean(), zone_df['m/s'].std()

    # Calculate number of observations for given number of years
    total_hours = num_years * 365.25 * 24
    num_observations = int(total_hours / timediv_mean)
    
    # Set the starting datetime to January 1st, 2000, 00:00:00
    current_datetime = datetime(2000, 1, 1, 0, 0, 0)

    # Initialize the new dataframe and generate the data
    simulated_df = pd.DataFrame(index=range(num_observations))
    simulated_df['timediv h'] = gamma(*timediv_params).rvs(size=num_observations).round(0)
    simulated_df['datetime'] = simulated_df['timediv h'].cumsum().apply(lambda x: current_datetime + timedelta(hours=x))
    simulated_df['kg'] = beta(*kg_params).rvs(size=num_observations).round(0)
    simulated_df['m/s'] = norm(ms_mean, ms_std).rvs(size=num_observations).round(0)
    simulated_df['kj'] = 0.5 * simulated_df['kg'] * (simulated_df['m/s']**2) /1000
    
    return simulated_df


In [ ]:
# Generate simulated dataframes
simulated1 = generate_simulated_data(zone1)
simulated1['zone'] = 1
simulated2 = generate_simulated_data(zone2)
simulated2['zone'] = 2
simulated1.describe()
# m/s is the min 0 which is not possible

In [ ]:
pd.concat([zone1, simulated1], axis=1, keys=['zone1', 'simulated1']).describe()

In [ ]:
simulated2.head()

In [ ]:
simulated2.tail()

In [ ]:
# Plot the histograms for 'kg', 'm/s', and 'timediv h'
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize=(10, 10))

# Histogram for 'kg'
ax1.hist(zone1['kg'], bins=20, color='red', alpha=0.5, label='Zone 1', density=True)
ax1.hist(simulated1['kg'], bins=20, color='orange', alpha=0.5, label='Simulated 1', density=True)
ax2.hist(zone2['kg'], bins=20, color='blue', alpha=0.5, label='Zone 2', density=True)
ax2.hist(simulated2['kg'], bins=20, color='green', alpha=0.5, label='Simulated 2', density=True)

ax1.set_xlabel('kg')
ax1.set_ylabel('Frequency Density')
ax1.legend()
ax2.set_xlabel('kg')
ax2.legend()

# Histogram for 'm/s'
ax3.hist(zone1['m/s'], bins=20, color='red', alpha=0.5, label='Zone 1', density=True)
ax3.hist(simulated1['m/s'], bins=20, color='orange', alpha=0.5, label='Simulated 1', density=True)
ax4.hist(zone2['m/s'], bins=20, color='blue', alpha=0.5, label='Zone 2', density=True)
ax4.hist(simulated2['m/s'], bins=20, color='green', alpha=0.5, label='Simulated 2', density=True)

ax3.set_xlabel('m/s')
ax3.set_ylabel('Frequency Density')
ax3.legend()
ax4.set_xlabel('m/s')
ax4.legend()

# Histogram for 'timediv h'
ax5.hist(zone1['timediv h'], bins=20, color='red', alpha=0.5, label='Zone 1', density=True)
ax5.hist(simulated1['timediv h'], bins=20, color='orange', alpha=0.5, label='Simulated 1', density=True)
ax6.hist(zone2['timediv h'], bins=20, color='blue', alpha=0.5, label='Zone 2', density=True)
ax6.hist(simulated2['timediv h'], bins=20, color='green', alpha=0.5, label='Simulated 2', density=True)

ax5.set_xlabel('timediv h')
ax5.set_ylabel('Frequency Density')
ax5.legend()
ax6.set_xlabel('timediv h')
ax6.legend()

plt.show()


In [ ]:
# Determine the latest end datetime of the two dataframes
max_datetime = min(simulated1['datetime'].max(), simulated2['datetime'].max())

# Set the end datetime of both dataframes to be the same
simulated1 = simulated1[simulated1['datetime'] <= max_datetime]
simulated2 = simulated2[simulated2['datetime'] <= max_datetime]

# Merge the two dataframes together, sort by datetime, and reset the index
simulated_df = pd.concat([simulated1, simulated2])
simulated_df = simulated_df.sort_values('datetime')
simulated_df = simulated_df.reset_index(drop=True)

In [ ]:
simulated_df.describe()

In [ ]:
simulated_df.tail(20)

because the reaction time is 24h we will asume that the nets will get emptied every evening if there are stones in it.


In [ ]:
# add a column that calculates the cumulative kg already in the net.

# first group the data by date
grouped_df = simulated_df.groupby(simulated_df['datetime'].dt.date)

# then calculate the cumulative sum of 'kg' within each group 
simulated_df['cumulative_kg'] = grouped_df['kg'].cumsum()
# and subtract the 'kg' valueof the new stone to get the weight in the net
simulated_df['cumulative_kg'] =  simulated_df['cumulative_kg'] - simulated_df['kg']

In [ ]:
# should we disregard the rest stones of the day if the net broke trough?
# after this the road probably gets closed

# Add a new column 'breakthrough'
simulated_df['breakthrough'] = 0

# Set breakthrough to 1 where conditions are met
condition1 = (simulated_df['kj'] > 1000)
condition2 = (simulated_df['cumulative_kg'] > 2000) & (simulated_df['kj'] > 500)
simulated_df.loc[condition1 | condition2, 'breakthrough'] = 1

In [ ]:
# here i look at the tail to make shure the cumulative_kg got calculated correctly.

simulated_df.tail(20)

In [ ]:
# Calculate the probability of a breakthrough
first_day = simulated_df['datetime'].min().date()
last_day = simulated_df['datetime'].max().date()
num_days = (last_day - first_day).days + 1

breaktroughs_prbability = (simulated_df['breakthrough'] == 1).sum() / num_days
breaktroughs_prbability

In [ ]:
simulated_df['breakthrough'].value_counts()

In [ ]:
# a 5m car driving 60 will be in this zone for: 
velocity = 60 / 3.6
print('velocity:', velocity, 'm/s')
danger_time = 5 / velocity
print('danger time: ', danger_time, 's')
# with 1200 cars a day this will be that amount of seconds in danger:
total_danger_time = 1200 * danger_time
print('total danger time:', total_danger_time, 's')
# precentage of cars being in danger per day:
danger_time_proportion = total_danger_time / (24 * 60 * 60)
print('danger time proportion: ', danger_time_proportion * 100, '%')

# how likely is it that a car will be in danger and the net will break trough?
dead_probability = breaktroughs_prbability * danger_time_proportion
print('dead probability:', dead_probability * 100, '%')